In [179]:
from nltk.tokenize import RegexpTokenizer
import numpy as np

In [181]:
import pandas as pd
df = pd.read_csv("./tripadvisor_hotel_reviews.csv")
df_100 = df.head(100)
df_100

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5
...,...,...
95,"excellent stay staff friendly helpful, nice ho...",4
96,return going seattle booked hotel knowing budg...,4
97,terrible hotel approximately 2 weeks ago april...,1
98,great price okay experience stayed inn queen a...,3


In [182]:
df_100.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  100 non-null    object
 1   Rating  100 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [279]:
df_100.describe().T

,count,mean,std,min,25%,50%,75%,max
Rating,100.0,3.63,1.338286,1.0,3.0,4.0,5.0,5.0


In [184]:
df_100.isnull().values.any()

False

In [185]:
df_100.columns

Index(['Review', 'Rating'], dtype='object')

In [187]:
def eleminatingStopWords(text):
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english')) 
    data = []
    for i in text:
        if i not in stop_words:
            data.append(i)
    return data

In [188]:
tokenizedReview = []
for sentence in review:
    each_word = RegexpTokenizer(r'\w+').tokenize(sentence)
    each_word = eleminatingStopWords(each_word)
    tokenizedReview.append(each_word)

tokenizedReview

[['nice',
  'hotel',
  'expensive',
  'parking',
  'got',
  'good',
  'deal',
  'stay',
  'hotel',
  'anniversary',
  'arrived',
  'late',
  'evening',
  'took',
  'advice',
  'previous',
  'reviews',
  'valet',
  'parking',
  'check',
  'quick',
  'easy',
  'little',
  'disappointed',
  'non',
  'existent',
  'view',
  'room',
  'room',
  'clean',
  'nice',
  'size',
  'bed',
  'comfortable',
  'woke',
  'stiff',
  'neck',
  'high',
  'pillows',
  'soundproof',
  'like',
  'heard',
  'music',
  'room',
  'night',
  'morning',
  'loud',
  'bangs',
  'doors',
  'opening',
  'closing',
  'hear',
  'people',
  'talking',
  'hallway',
  'maybe',
  'noisy',
  'neighbors',
  'aveda',
  'bath',
  'products',
  'nice',
  'goldfish',
  'stay',
  'nice',
  'touch',
  'taken',
  'advantage',
  'staying',
  'longer',
  'location',
  'great',
  'walking',
  'distance',
  'shopping',
  'overall',
  'nice',
  'experience',
  'pay',
  '40',
  'parking',
  'night'],
 ['ok',
  'nothing',
  'special',
  

## Levenshtein Distance

In [189]:
def levenshteinDistanceDP(token1, token2, printFunc = False):
    distances = np.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1
    if printFunc:
        for t1 in range(len(token1) + 1):
            for t2 in range(len(token2) + 1):
                print(int(distances[t1][t2]), end=" ")
            print()
    if len(token1) > len(token2):
        return 1 - (distances[len(token1)][len(token2)] / len(token1))
    else:
        return 1 - (distances[len(token1)][len(token2)] / len(token2))
    # return distances[len(token1)][len(token2)] //DEFAULT LEVENSHTEIN DISTANCE RETURN

In [206]:
levenshteinDistanceDP("naber", "ne haber")

0.625

In [207]:
levenshteinDistanceDP("abcçdef", "bcçdef")

0.8571428571428572

In [246]:
def findSimilar(dataSet, target):
    maxSimilarityWieight = 0
    similarSentence = ""
    for sentences in dataSet:
        for element1 in sentences:        
            weightArray = []
            for element2 in target:
                weight = levenshteinDistanceDP(element1, element2)
                weightArray.append(weight)
            meanWeight = np.mean(np.array(weightArray))
            if  meanWeight > maxSimilarityWieight:
                maxSimilarityWieight = meanWeight
                similarSentence = sentences
    for i in range(len(tokenizedReview)):
        if tokenizedReview[i] == similarSentence:
            return  "Similarity Weight =", maxSimilarityWieight, ", Rating =", rate[i], ", Sentence = ", similarSentence

In [292]:
findSimilar(tokenizedReview, ["this", "place", "is", "awesome"])

('Similarity Weight =',
 0.3571428571428572,
 ', Rating =',
 5,
 ', Sentence = ',
 ['great',
  'stay',
  'great',
  'stay',
  'went',
  'seahawk',
  'game',
  'awesome',
  'downfall',
  'view',
  'building',
  'n',
  'complain',
  'room',
  'huge',
  'staff',
  'helpful',
  'booked',
  'hotels',
  'website',
  'seahawk',
  'package',
  'charge',
  'parking',
  'got',
  'voucher',
  'taxi',
  'problem',
  'taxi',
  'driver',
  'n',
  'want',
  'accept',
  'voucher',
  'barely',
  'spoke',
  'english',
  'funny',
  'thing',
  'speak',
  'arabic',
  'called',
  'started',
  'making',
  'comments',
  'girlfriend',
  'cell',
  'phone',
  'buddy',
  'took',
  'second',
  'realize',
  'said',
  'fact',
  'speak',
  'language',
  'face',
  'priceless',
  'ass',
  'told',
  'said',
  'large',
  'city',
  'told',
  'head',
  'doorman',
  'issue',
  'called',
  'cab',
  'company',
  'promply',
  'answer',
  'n',
  'apologized',
  'offered',
  'pay',
  'taxi',
  'bucks',
  '2',
  'miles',
  'stadi

In [288]:
tokenizedReviewExample = []
each_word_example = RegexpTokenizer(r'\w+').tokenize(df[["Review"][0]].values[105])
each_word_example = eleminatingStopWords(each_word_example)
tokenizedReviewExample.append(each_word_example)
tokenizedReviewExample = tokenizedReviewExample[0]
tokenizedReviewExample

['good',
 'choice',
 'seattle',
 'said',
 'little',
 'gem',
 'boutique',
 'hotel',
 'edge',
 'downtown',
 'old',
 'building',
 'nicely',
 'maintained',
 'restored',
 'people',
 'like',
 'slightly',
 'old',
 'finish',
 'loved',
 'room',
 'floor',
 'building',
 'meant',
 'quiet',
 'fairly',
 'clear',
 'view',
 'space',
 'needle',
 '5',
 'blocks',
 'away',
 'room',
 '2',
 'queen',
 'size',
 'beds',
 'separate',
 'small',
 'sitting',
 'room',
 'kitchen',
 'worked',
 'clean',
 'staff',
 'friendly',
 'professional',
 'valet',
 'parking',
 'quick',
 'easy',
 'location',
 'partial',
 'residential',
 'area',
 'homely',
 'feel',
 'outside',
 'lots',
 'good',
 'neighbourhood',
 'restaurants',
 'pubs',
 'short',
 'walk',
 'seattle',
 'center',
 'provided',
 'access',
 'monorail',
 'quck',
 'trip',
 'downtown',
 'definitely',
 'stay',
 'seattle']

In [289]:
df[["Rating"][0]].values[105]

4

In [290]:
findSimilar(tokenizedReview, tokenizedReviewExample)

('Similarity Weight =',
 0.20250796212334668,
 ', Rating =',
 4,
 ', Sentence = ',
 ['excellent',
  'staff',
  'housekeeping',
  'quality',
  'hotel',
  'chocked',
  'staff',
  'make',
  'feel',
  'home',
  'experienced',
  'exceptional',
  'service',
  'desk',
  'staff',
  'concierge',
  'door',
  'men',
  'maid',
  'service',
  'needs',
  'work',
  'maid',
  'failed',
  'tuck',
  'sheets',
  'foot',
  'bed',
  'instance',
  'soiled',
  'sheets',
  'used',
  'staff',
  'quickley',
  'resolved',
  'soiled',
  'sheets',
  'issue',
  'guess',
  'relates',
  'employee',
  'reflection',
  'rest',
  'staff',
  'received',
  'excellent',
  'advice',
  'concierge',
  'regarding',
  'resturants',
  'area',
  'happy',
  'hour',
  'wine',
  'tasting',
  'nice',
  'touch',
  'staff',
  'went',
  'way',
  'make',
  'feel',
  'home',
  'great',
  'location',
  'like',
  'close',
  'good',
  'food',
  'shopping',
  'took',
  'play',
  '5th',
  'street',
  'theather',
  'well',
  'pikes',
  'market',